# Quickstart

This notebook is just to make sure that everything works. Firstly, let's pull the latest changes for this repo.


In [ ]:
!git pull

In [ ]:
#Let's install the requirements
!pip3 install -r ../requirements.txt

In [ ]:
import os
import tiktoken
import openai
from dotenv import load_dotenv
import sys

# Load environment variables
load_dotenv('../.env')

# Option 2 - Using Access Key
openai.api_type = "azure"
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_version = os.environ.get('OPENAI_API_VERSION', "2022-12-01")

# Define embedding model and encoding
EMBEDDING_MODEL = os.environ.get('OPENAI_EMBEDDING_MODE', 'text-embedding-ada-002')
COMPLETION_MODEL = os.environ.get('OPENAI_COMPLETION_MODEL', 'gpt-35-turbo')
encoding = tiktoken.get_encoding('cl100k_base')

Test if tokenizer works:

In [ ]:
text_to_encode = "Hello world!"
tokens = encoding.encode(text_to_encode)
print(f"There are {len(tokens)} for text prompt: '{text_to_encode}'")
print(tokens)

Test if we can reach OpenAI

In [ ]:
prompt = 'Answer shortly "Who is the prime minister of Israel?"'  # The prompt to generate completions for
# Generate 3 completions
response = openai.Completion.create(engine="gpt-35-turbo",
                                    prompt=prompt,
                                    temperature=0,
                                    max_tokens=10,
                                    top_p=1
                                    )  # Change the temperature to generate more or less random completions
answer = response.choices[0].text
print(f"{answer}")


Do it in a streaming fashion from OpenAI

In [ ]:
for resp in openai.Completion.create(
    engine='gpt-35-turbo', 
    prompt='Give me 5 taglines for an ice cream shop', 
    max_tokens=512, 
    stream=True,
    temperature=0.7):
    sys.stdout.write(resp.choices[0].text)
    sys.stdout.flush()

To save costs we would like to use a local embedding code instead of working with API. Lets test if Local embedding model work:

In [ ]:
from sentence_transformers import SentenceTransformer

sentences = ['Hello World!']
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(f"Got embeddings with shape {embeddings.shape}")


Compare the embedding length coming from OpenAi's remote model (text-embedding-ada-002):

In [ ]:
#Use it only once and then the local embeddings.
response = openai.Embedding.create(input="Hello World!", engine=EMBEDDING_MODEL)
e = response["data"][0]["embedding"]
print("Embedding shape", len(e))


print('usage', response['usage'])
print('model', response['model'])
print("Full response keys from embedding", response.keys())
